#1 Introducción


Suma de vectores usando la GPU usando paralelismo


#2 Armado del ambiente

Instala en el cuaderno el módulo CUDA de Python.

In [ ]:
!pip install pycuda

#3 Desarrollo

Versión secuencial CPU

In [47]:
from datetime import datetime

lista1 = [1,2,3,4,5]
lista2 = [1,2,3,4,5]
lista3 = [0,0,0,0,0]

tiempoInicialFor = datetime.now()
for i in range(5):
  lista3[i] = lista1[i] + lista2[i]

tiempoFinalFor = datetime.now()

tiempoFor = tiempoFinalFor.microsecond - tiempoInicialFor.microsecond
print(lista3);
print(tiempoFor)

[2, 4, 6, 8, 10]
76


Versión con paralelismo GPU

In [ ]:
import pycuda.driver as cuda
import pycuda.autoinit
from   pycuda.compiler import SourceModule

from  datetime import datetime
import numpy

n=50
lista1 = numpy.random.randn(n)
lista1 = lista1.astype(numpy.float32())

lista2 = numpy.random.rand(n)
lista2 = lista2.astype(numpy.float32())

lista3 = numpy.empty_like(lista1)

# CPU - reservo la memoria GPU
lista1_gpu = cuda.mem_alloc(lista1.nbytes)
lista2_gpu = cuda.mem_alloc(lista2.nbytes)

# GPU - copio la memoria al GPU
cuda.memcpy_htod(lista1_gpu,lista1)
cuda.memcpy_htod(lista2_gpu,lista2)

#CPU - Defino la funcion kernel que ejecutará en GPU
module = SourceModule("""
__global__ void kernel_sumaListas(int n, float *plista1 , float *plista2)
{
    int idx = threadIdx.x + blockIdx.x*blockDim.x;
    if(idx<n)
    {
      plista2[idx] += plista1[idx];
    }
}

""")

# CPU - Genero la funcion kernel
kernel = module.get_function("kernel_sumaListas")

tiempoInicialGpu = datetime.now()

# GPU - Ejecuto el kernel
dim_hilo = 256
dim_bloque = numpy.int((n+dim_hilo-1)/dim_hilo)
print("Thread: ",dim_hilo, "Bloque: ", dim_bloque)

kernel(numpy.int32(n),lista1_gpu,lista2_gpu, block = (dim_hilo,1,1),grid = (dim_bloque,1,1))

tiempoFinalGpu = datetime.now()
#GPU - copio el resultado desde la memoria de la GPU
cuda.memcpy_dtoh(lista3,lista2_gpu)

tiempoFinal = tiempoFinalGpu.microsecond - tiempoInicialGpu.microsecond
print(lista3);
print("Tiempo total: ",tiempoFinal,"[ms]")

#4 Tabla de pasos

#5 Conclusiones

#6 Bibliografia